## Line Detections

### Image

In [15]:
import cv2
import numpy as np

# Cargar la imagen
image = cv2.imread('img/freeway.jpg')
# image = cv2.imread('img/board.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Aplicar desenfoque para reducir el ruido
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detección de bordes con Canny
edges = cv2.Canny(blurred, 50, 100)

# Aplicar la transformada de Hough para la detección de líneas
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Dibujar las líneas detectadas
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Mostrar la imagen con las líneas detectadas
cv2.imshow("Edges", edges)
cv2.imshow("Líneas detectadas", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [14]:
import cv2
import numpy as np

# Iniciar la captura de video desde la cámara
cap = cv2.VideoCapture(0)

while True:
    # Leer el frame de la cámara
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Aplicar desenfoque para reducir el ruido
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detección de bordes con Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Aplicar la transformada de Hough para la detección de líneas
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Dibujar las líneas detectadas en el frame
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Mostrar el frame con las líneas detectadas
    cv2.imshow("Líneas en tiempo real", frame)

    # Salir del loop si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import numpy as np

# Abrir el archivo de video
cap = cv2.VideoCapture('img/driving_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height = frame.shape[0]
    width = frame.shape[1]

    region_of_interest_vertices = [
        (0, height),
        (width / 2, height / 1.4), # 1.4 best value
        (width, height),
    ]
    
    # Convertir a escala de grises
    gray_image = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    

    gray_image = cv2.GaussianBlur(gray_image, (5,5), 0.1)
    cv2.imshow('Gray Image', gray_image)

    # Crear una imagen en color para dibujar los puntos de interés
    gray_with_points = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2RGB)

    colors = ((255,0,0), (0,255,0), (0,0,255))
    # Dibujar los puntos de interés en rojo
    for i, vertex in enumerate(region_of_interest_vertices):
        cv2.circle(gray_with_points, (int(vertex[0]), int(vertex[1])), 10, colors[i], -1)  # -1 rellena el círculo
    
    cv2.imshow('Gray Image with Points', gray_with_points)

    # Aplicar detección de bordes Canny
    canny_image = cv2.Canny(gray_image, 100, 120)
    cv2.imshow('Canny Image', canny_image)
    
    # Aplicar la región de interés
    mask = np.zeros_like(canny_image)
    cv2.fillPoly(mask, np.array([region_of_interest_vertices], np.int32), 255)
    cropped_image = cv2.bitwise_and(canny_image, mask)
    cv2.imshow('Cropped Image', mask)
    
    
    # Detectar líneas usando la Transformada de Hough
    lines = cv2.HoughLinesP(cropped_image,
                            rho=2,
                            theta=np.pi / 180,
                            threshold=50,
                            lines=np.array([]),
                            minLineLength=40,
                            maxLineGap=100)
    
    # Dibujar líneas en la imagen original
    img_with_lines = np.copy(frame)
    blank_image = np.zeros((img_with_lines.shape[0], img_with_lines.shape[1], 3), dtype=np.uint8)

    

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(blank_image, (x1, y1), (x2, y2), (0, 255, 0), thickness=3)

    cv2.imshow("blank", blank_image)
    
    img_with_lines = cv2.addWeighted(img_with_lines, 0.8, blank_image, 0.5, 0.0)
    
    # Mostrar el marco procesado
    cv2.imshow('Processed Frame', img_with_lines)
    
    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(10) == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
